In [ ]:
from logging import getLogger
from datetime import datetime
import pandas as pd
from btc_functions.logging.logger_config import setup_logger
import os
from dotenv import load_dotenv
import btc_functions.load_database.mysql as db_functions
from btc_functions.transfert_data.get_data_as_df import get_df_change_timestamp

: 

In [ ]:
def reverse_timestamp(df: pd.DataFrame, col1: str, col2: str = None) -> pd.DataFrame:
    """Attention, la fonction traite max 2 colonnes

    Args:
        df (pd.DataFrame): df
        col1 (str): nom de la col à transformer
        col2 (str, optional): nom de la col2 à transformer. Defaults to None.

    Returns:
        pd.DataFrame: df
    """
    cols = [col1] if col2 is None else [col1, col2]
    df[cols] = df[cols].map(lambda x: datetime.fromtimestamp(x / 1000))
    return df


def fetch_and_process_table(table_name, col1, col2 = None) -> pd.DataFrame:
    """Récupère une table SQL, applique reverse_timestamp sur les cols BIGINT spécifiées
       et retourne le DataFrame.

    Args:
        engine (connection): engine pymysql
        table_name (sql_table): nom de la table cible (type BIGINT / timestamp)
        col1 (table_column): table à transcrire
        col2 (table_column2, optional): 2e colonne à transcrire. Defaults to None.
    """
    load_dotenv("/home/sanou/BTC_app/env/private.env")
    engine = db_functions.create_connection()
    query = f"SELECT * FROM {table_name}"

    df = pd.read_sql_query(query, engine)
    df = reverse_timestamp(df, col1, col2)
    
    # logger.info(f"Table {table_name} traitée avec succès.")
    return df

In [ ]:
load_dotenv("/home/sanou/BTC_app/env/private.env")
table_klines = "klines"
table_24 = "ticket24h"
tables = ["klines", "ticket24h"]

engine = db_functions.create_connection()

for table in tables:
    query = f"SELECT * FROM {table}"
    df = pd.read_sql_query(query, engine)
    bigint_cols = df.select_dtypes(include=['int64']).columns
    
    for col in bigint_cols:
        df = reverse_timestamp(df, bigint_cols)
        print(f'{col} traitée.')
        print(df.head())


table_klines = "klines"
col_kline1 = "kline_open_time"
col_kline2 = "kline_close_time"
table_24 = "ticket24h"
col_ticket1 = "openTime"
col_ticket2 = "closeTime"

query_klines = f"SELECT * FROM {table_klines}"
query_ticket = f"SELECT * FROM {table_24}"
df_klines = pd.read_sql_query(query_klines, engine)
df_24 = pd.read_sql_query(query_ticket, engine)

df_klines = reverse_timestamp(df_klines, col_kline1, col_kline2)
df_24 = reverse_timestamp(df_24, col_ticket1, col_ticket2)


print(df_klines.head(), df_24.head())

In [4]:
load_dotenv("/home/sanou/BTC_app/env/private.env")
df_klines = get_df_change_timestamp("klines", "kline_open_time", "kline_close_time")
df_daily = get_df_change_timestamp("daily", "openTime", "closeTime")

2024-11-26 00:58:23,315 - btc_functions.load_database.mysql - INFO - Tentative de connexion à localhost:3306 en tant que sanou
2024-11-26 00:58:23,317 - btc_functions.load_database.mysql - INFO - Connexion à MySQL réussie avec SQLAlchemy.
2024-11-26 00:58:23,427 - btc_functions.transfert_data.get_data_as_df - INFO - Table klines traitée avec succès.
2024-11-26 00:58:23,428 - btc_functions.load_database.mysql - INFO - Tentative de connexion à localhost:3306 en tant que sanou
2024-11-26 00:58:23,429 - btc_functions.load_database.mysql - INFO - Connexion à MySQL réussie avec SQLAlchemy.
2024-11-26 00:58:23,440 - btc_functions.transfert_data.get_data_as_df - INFO - Table daily traitée avec succès.


In [7]:
# df_klines
# df_klines.head()
df_klines['kline_open_time'] = pd.to_datetime(df_klines['kline_open_time']).dt.floor('D')
df_klines.head()

,kline_open_time,open_price,high_price,low_price,close_price,volume,kline_close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,2024-11-01,70479.99,70618.63,70460.58,70524.90,139.70192,2024-11-01 00:04:59.999,9.856709e+06,14880,90.64178,6.395008e+06
1,2024-11-01,70524.90,70524.90,70402.00,70426.01,40.21647,2024-11-01 00:09:59.999,2.832940e+06,9612,13.47755,9.492771e+05
2,2024-11-01,70426.01,70499.99,70402.00,70496.46,48.62536,2024-11-01 00:14:59.999,3.426002e+06,5507,31.00472,2.184579e+06
3,2024-11-01,70496.47,70586.00,70494.20,70579.99,35.46521,2024-11-01 00:19:59.999,2.501872e+06,10021,19.37823,1.367012e+06
4,2024-11-01,70580.00,70624.00,70561.83,70568.12,74.17420,2024-11-01 00:24:59.999,5.236623e+06,7532,44.11568,3.114561e+06


In [15]:
# df_24
col = "openTime"
df_daily['openTime'] = pd.to_datetime(df_daily['openTime']).dt.floor('D')
df_daily[col] = df_daily[col] - pd.Timedelta(days=1)
df_daily.head()

,symbol,priceChange,priceChangePercent,weightedAvgPrice,openPrice,highPrice,lowPrice,lastPrice,volume,quoteVolume,openTime,closeTime,firstId,lastId,count
0,BTCUSDT,-128.54,-0.13,98549.963340,98892.0,98908.85,98300.00,98763.46,3642.43066,3.589614e+08,2024-11-22,2024-11-24 00:59:59.999,4132567696,4133037760,470065
1,BTCUSDT,751.59,0.77,98113.046908,97672.4,98500.00,97620.74,98423.99,2770.48968,2.718212e+08,2024-11-23,2024-11-25 00:59:59.999,4136406834,4136757274,350441


In [9]:
merged_df = pd.merge(df_klines, df_daily, 
                     left_on="kline_open_time",
                     right_on="openTime",
                     how="inner")

In [10]:
features_columns = [
    "open_price",
    "high_price",
    "low_price",
    "close_price",
    "volume_x",
    "priceChange",
    "priceChangePercent",
]

X = merged_df[features_columns].copy()
X["price_change"] = X["close_price"].shift(-1) - merged_df["close_price"]
X["trend"] = (X["price_change"] > 0).astype(int)
y = X["trend"]
X = X.drop(["trend", "price_change"], axis=1)

X.head()

,open_price,high_price,low_price,close_price,volume_x,priceChange,priceChangePercent
0,98627.36,98822.03,98627.35,98815.98,81.53313,-128.54,-0.13
1,98815.97,98815.98,98640.00,98710.12,114.15826,-128.54,-0.13
2,98710.12,98871.80,98710.12,98811.03,77.55088,-128.54,-0.13
3,98811.03,98811.04,98730.00,98755.48,57.00431,-128.54,-0.13
4,98755.48,98775.61,98742.39,98756.60,36.12271,-128.54,-0.13


In [11]:
y.head()

0    0
1    1
2    0
3    1
4    0
Name: trend, dtype: int64

In [20]:
merged_df.to_csv("/home/sanou/BTC_app/1_python_scripts/test_merge.csv")

In [ ]:
df_klines.to_csv("/home/sanou/BTC_app/1_python_scripts/test_kline.csv")
df_24.to_csv("/home/sanou/BTC_app/1_python_scripts/test_df24.csv")

########################################    

In [ ]:
from logging import getLogger
import os
from glob import glob
import pandas as pd
from joblib import load
from btc_functions.logging.logger_config import setup_logger
import btc_functions.extract_data.binance_dailies as bd
import btc_functions.transfert_data.best_model as bm
from btc_functions.load_database.mysql import convert_all_json_to_csv
from sklearn.preprocessing import StandardScaler

# logger = getLogger(__name__)
# setup_logger()

model_path = "/home/sanou/BTC_app/models_ml/best_LinearRegression.pickle"
model = load(model_path)

json_dir = "/home/sanou/BTC_app/data/1_raw"
csv_dir = "/home/sanou/BTC_app/data/1_raw"

today_data_json = bd.get_data_from_binance("klines", 0)
convert_all_json_to_csv(json_dir, csv_dir)
csv_kline = glob(os.path.join(csv_dir, "*KLINES.csv"))

if csv_kline:
    df_list = [pd.read_csv(file) for file in csv_kline]
    df = pd.concat(df_list, ignore_index=True)
    print(df.head())
else:
    print("error DF LUL")
    exit(1)

if not df.empty:
    # Formatage des données
    df_formatted = bm.format_time_ml(df, "kline_open_time")
    X, y = bm.prepare_data(df_formatted)

    # Normalisation si le modèle est LinearRegression
    if "LinearRegression" in model_path:
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        print("Data normalized.")
    else:
        X_scaled = X

    # Prédictions
    y_pred = model.predict(X_scaled)
    df_formatted["Predict Trend"] = (y_pred > 0.5).astype(int)

    # Sauvegarde des résultats
    output_file = "/home/sanou/BTC_app/models_ml/latest_predictions_with_trend.csv"
    df_formatted.to_csv(output_file, index=False)
    print(f"Predictions saved to {output_file}.")
else:
    print("DataFrame is empty. Cannot proceed with predictions.")





   kline_open_time  open_price  high_price  low_price  close_price     volume  \
0    1733180400000    95372.00    95529.19   95328.00     95507.94  113.49153   
1    1733180700000    95507.93    95603.75   95484.77     95532.49  111.35445   
2    1733181000000    95532.48    95584.00   95464.01     95516.47   92.05291   
3    1733181300000    95516.48    95667.75   95516.47     95572.56   69.75822   
4    1733181600000    95572.56    95576.10   95496.98     95533.37   81.27548   

   kline_close_time  quote_asset_volume  number_of_trades  \
0     1733180699999        1.083141e+07             20591   
1     1733180999999        1.063956e+07             11807   
2     1733181299999        8.793703e+06             11829   
3     1733181599999        6.668949e+06             11456   
4     1733181899999        7.765189e+06             10207   

   taker_buy_base_asset_volume  taker_buy_quote_asset_volume  
0                     80.63604                  7.695875e+06  
1                   

KeyError: "['volume_x', 'priceChange', 'priceChangePercent'] not in index"